In [1]:

import numpy as np
import cv2
Thresh = 1000000
import random
import os
i1 = 1
sizeofBOVW = 32
Epochs = 100
def build_hist(data):
    arr = [0]*8
    for i in data:
        arr[i//32] += 1
    return arr

def Build_hist_for_patch(img_data,row,column):
    red = []
    green = []
    blue = []
    for i in range(row,row+32):
        for j in range(column,column+32):
            red.append(img_data[i][j][0])
            green.append(img_data[i][j][1])
            blue.append(img_data[i][j][2])
    hist = build_hist(red)+build_hist(green)+build_hist(blue)
    return np.array(hist)


def img_feature_extraction(img_data):
    img = []
    for row in range(0,len(img_data),32):
        for col in range(0,len(img_data[0]),32):
            img.append(Build_hist_for_patch(img_data,row,col))
    return img

def BOVW(data):
    BOVWDATA = [np.zeros(32) for i in range(len(data))]
    for i in range(len(data)):
        for j in range(len(data[i])):
            BOVWDATA[i][data[i][j]] += 1
        BOVWDATA[i] /= len(data[i])
    return BOVWDATA

def ExtractFeatureVector(img_data,clusterPoints):
    img_data = cv2.copyMakeBorder(img_data, 0,(32-len(img_data)%32)%32 , 0, (32-len(img_data[0])%32)%32, cv2.BORDER_REFLECT)
    img_data = img_feature_extraction(img_data)
    FeatureVector = np.zeros(32)
    for i in range(len(img_data)):
        FeatureVector[np.argmin([sum((img_data[i]-clusterPoints[k])**2) for k in range(len(clusterPoints))])] += 1
    FeatureVector /= len(img_data)
    return FeatureVector

def KmeansClustering(data,prevclasses,clusterPoints):
    classes = [[-1]*len(data[i]) for i in range(len(data))]
    updateCount = 0
    DistMeas = 0
    SumNewClusterPoints = [np.zeros(24) for i in range(32)]
    CountNewClusterPoints = [0]*32
    for i in range(len(data)):
        for j in range(len(data[i])):
            classes[i][j] = np.argmin([sum((data[i][j]-clusterPoints[k])**2) for k in range(len(clusterPoints))])
            if classes[i][j] != prevclasses[i][j]:
                updateCount += 1
            DistMeas += sum((data[i][j] - clusterPoints[classes[i][j]])**2)
            CountNewClusterPoints[classes[i][j]] += 1
            SumNewClusterPoints[classes[i][j]] += data[i][j]
    NewClusterPoints = [SumNewClusterPoints[i]/CountNewClusterPoints[i] if CountNewClusterPoints[i]!=0 else np.zeros(24) for i in range(32)]
    print(DistMeas)
    global Epochs
    Epochs -= 1
    if updateCount == 0 or Epochs == 0:
        return (NewClusterPoints,BOVW(classes))
    else:
        return KmeansClustering(data,classes,NewClusterPoints)
    
p_dir = "./Group02/Classification/Image_Group02/test"
coast = os.path.join(p_dir,"coast")
industrial_area = os.path.join(p_dir,"industrial_area")
pagoda = os.path.join(p_dir,"pagoda")
data = []
TrainClasses = []
for i in os.listdir(coast):
    img_data = cv2.imread(os.path.join(coast,i))
    img_data = cv2.copyMakeBorder(img_data, 0,(32-len(img_data)%32)%32 , 0, (32-len(img_data[0])%32)%32, cv2.BORDER_REFLECT)
    img_data = img_feature_extraction(img_data)
    data.append(img_data)
    TrainClasses.append(0)
for i in os.listdir(industrial_area):
    img_data = cv2.imread(os.path.join(industrial_area,i))
    img_data = cv2.copyMakeBorder(img_data, 0,(32-len(img_data)%32)%32 , 0, (32-len(img_data[0])%32)%32, cv2.BORDER_REFLECT)
    img_data = img_feature_extraction(img_data)
    data.append(img_data)
    TrainClasses.append(1)
for i in os.listdir(pagoda):
    img_data = cv2.imread(os.path.join(pagoda,i))
    img_data = cv2.copyMakeBorder(img_data, 0,(32-len(img_data)%32)%32 , 0, (32-len(img_data[0])%32)%32, cv2.BORDER_REFLECT)
    img_data = img_feature_extraction(img_data)
    data.append(img_data)
    TrainClasses.append(2)
    

classes = [[-1]*len(data[i]) for i in range(len(data))]
ClusterPoints = []
for i in range(32):
    p = random.randint(0,len(data)-1)
    ClusterPoints.append(data[p][random.randint(0,len(data[p])-1)])
BOVW_DATA = KmeansClustering(data,classes,ClusterPoints)
ClusterPoints = BOVW_DATA[0]
Image_Data = BOVW_DATA[1]

64206202748
45969113750.488174
42527449926.22031
41394953913.65578
40639250797.593895
39813224922.57683
39178569520.51054
38719137695.42162
38387543852.77885
38168904971.3019
37991959460.418526
37575482259.41238
37431836952.246994
37186668047.81418
36164743284.3877
35808865287.275566
35542890412.02686
35197499678.80083
34831239858.56235
34648847349.0458
34605617325.13861
34574273674.8557
34549623078.130585
34530394213.1481
34515328340.93689
34503286112.82886
34492943590.19164
34483930410.446495
34474808661.30562
34466801718.633705
34458982834.22294
34450738957.4576
34443329674.904305
34436340013.04893
34428110619.35443
34419125271.020424
34411725074.625984
34405649397.49633
34400647006.009155
34395937085.00253
34391884419.3049
34388097105.84494
34383867418.46005
34378377980.21529
34368052912.73244
34357341059.783222
34348337988.516457
34342235287.723003
34337546470.31405
34332786800.141834
34328250165.935333
34323497965.189754
34318816767.131607
34314307252.521046
34309891317.271534
34

In [36]:
Image_data
    
    

NameError: name 'Image_data' is not defined

In [2]:
np.savetxt('image_test.txt', np.array(Image_Data))

In [3]:
np.savetxt('image_test_label.txt', np.array(TrainClasses))


In [5]:
Image_Data

[array([0.        , 0.01754386, 0.03508772, 0.14035088, 0.        ,
        0.        , 0.        , 0.        , 0.01754386, 0.00701754,
        0.01403509, 0.05964912, 0.09122807, 0.03508772, 0.01052632,
        0.01754386, 0.        , 0.0877193 , 0.03859649, 0.01403509,
        0.02105263, 0.        , 0.01052632, 0.01403509, 0.05964912,
        0.        , 0.00350877, 0.09122807, 0.07017544, 0.07719298,
        0.05614035, 0.01052632]),
 array([0.        , 0.00368421, 0.01894737, 0.04263158, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.00421053,
        0.03315789, 0.00157895, 0.06578947, 0.01315789, 0.10631579,
        0.        , 0.25684211, 0.00315789, 0.        , 0.00105263,
        0.00684211, 0.        , 0.01421053, 0.10473684, 0.00684211,
        0.04421053, 0.        , 0.01842105, 0.02263158, 0.        ,
        0.01473684, 0.21684211]),
 array([0.      , 0.03125 , 0.      , 0.      , 0.      , 0.03125 ,
        0.015625, 0.      , 0.      , 0.015625, 